In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)


In [3]:
txt = open('notshy_talk.txt', encoding='utf-8').read()

In [4]:
txt_split = pd.Series(txt.split("\n")) #줄바꿈 기준 나누기

In [5]:
txt_split[:10] #채팅 내용이 아닌 초대/날짜 등의 텍스트 존재

0                                  Not shy🦠 님과 카카오톡 대화
1                         저장한 날짜 : 2020-09-09 00:59:15
2                                                     
3     --------------- 2020년 8월 22일 토요일 ---------------
4    곽재원님이 김은영님, 김은영님, 이하빕 19글인님, 김은영님, 이하빕 19글인님, ...
5                                  [재오닝] [오후 9:13] 열어분
6                              [재오닝] [오후 9:13] 안 부담스럽게
7           [재오닝] [오후 9:13] 우리끼리 연습영상 정도만 찍는거 어때여?!?!?
8                                 [재오닝] [오후 9:13] 이모티콘
9                        [수빈] [오후 9:13] 조 아 요 ~!~!~!~!
dtype: object

In [6]:
import re

In [7]:
txt_split[13] #이름에 숫자 있는 경우 있음

'[이하빕 19글인] [오후 9:20] 조아용 !'

In [8]:
txt_split[20] #이름에 이모지 있는 경우 있음

'[소정먼지🌬] [오전 12:32] 안농'

In [9]:
df = pd.DataFrame(columns=["time","name","chat"])
df

,time,name,chat


In [10]:
time = re.compile(r"오. \d{1,2}:\d{2}") #time을 "오" + (문자 1개)(전or후) + (공백) + (숫자 2개) + ":" + (숫자 2개) 의 형태로 추출
time.search(txt_split[10]).group()

'오후 9:13'

In [11]:
name = re.compile(r"\[\w+\s?\w+[\U00010000-\U0010ffff]?\]") #이름 - 대괄호와 그 사이에 문자,숫자,공백 있을 수 있지만 기호는 포함하지 않는 기준으로
                                                            #[\U00010000-\U0010ffff]? 이모지 있을수도 있음
name.search(txt_split[10]).group()

'[수빈]'

In [12]:
for i in range(len(txt_split)):
    if (time.search(txt_split[i]) is not None) and (name.search(txt_split[i]) is not None): #time과 name 모두 있는 항목에서
                                                                                            #(채팅내용 아닌 항목 구분하기 위해)
        df2 = pd.DataFrame({"time": [time.search(txt_split[i]).group()], 
                            "name": [name.search(txt_split[i]).group()], 
                            "chat": [txt_split[i]]}) #우선 채팅내용은 텍스트 전체
        df = df.append(df2,ignore_index=True)
    else:
        continue
df.head()

,time,name,chat
0,오후 9:13,[재오닝],[재오닝] [오후 9:13] 열어분
1,오후 9:13,[재오닝],[재오닝] [오후 9:13] 안 부담스럽게
2,오후 9:13,[재오닝],[재오닝] [오후 9:13] 우리끼리 연습영상 정도만 찍는거 어때여?!?!?
3,오후 9:13,[재오닝],[재오닝] [오후 9:13] 이모티콘
4,오후 9:13,[수빈],[수빈] [오후 9:13] 조 아 요 ~!~!~!~!


In [13]:
#이름 대괄호 기준으로 같이 추출했으므로 대괄호 삭제 
for i in range(len(df)):
    df.iloc[i]["name"] = df.iloc[i]["name"][1:-1] 
df.head()

,time,name,chat
0,오후 9:13,재오닝,[재오닝] [오후 9:13] 열어분
1,오후 9:13,재오닝,[재오닝] [오후 9:13] 안 부담스럽게
2,오후 9:13,재오닝,[재오닝] [오후 9:13] 우리끼리 연습영상 정도만 찍는거 어때여?!?!?
3,오후 9:13,재오닝,[재오닝] [오후 9:13] 이모티콘
4,오후 9:13,수빈,[수빈] [오후 9:13] 조 아 요 ~!~!~!~!


In [14]:
#chat 에 있는 전체 텍스트에서 시간 제거
for i in range(len(df)):
    df.iloc[i]["chat"] = df.iloc[i]["chat"].replace("["+time.search(df.iloc[i]["chat"]).group()+"]","")
df.head()

,time,name,chat
0,오후 9:13,재오닝,[재오닝] 열어분
1,오후 9:13,재오닝,[재오닝] 안 부담스럽게
2,오후 9:13,재오닝,[재오닝] 우리끼리 연습영상 정도만 찍는거 어때여?!?!?
3,오후 9:13,재오닝,[재오닝] 이모티콘
4,오후 9:13,수빈,[수빈] 조 아 요 ~!~!~!~!


In [15]:
#chat 에 있는 전체 텍스트에서 이름 제거 + 앞뒤 공백 제거
for i in range(len(df)):
    df.iloc[i]["chat"] = df.iloc[i]["chat"].replace(name.search(df.iloc[i]["chat"]).group(),"").lstrip()
df

,time,name,chat
0,오후 9:13,재오닝,열어분
1,오후 9:13,재오닝,안 부담스럽게
2,오후 9:13,재오닝,우리끼리 연습영상 정도만 찍는거 어때여?!?!?
3,오후 9:13,재오닝,이모티콘
4,오후 9:13,수빈,조 아 요 ~!~!~!~!
5,오후 9:13,수빈,낫샤이 뒤에는
6,오후 9:13,수빈,코로나인가요?
7,오후 9:14,재오닝,맞아요~
8,오후 9:20,이하빕 19글인,조아용 !
9,오후 9:43,김은영,우왕! 조아욤
